# Determining a Location for a New Restaurant in Troy, Alabama USA

   In order to determine a good location for a restaurant in Troy AL. I am going to first split the city into it's three zipcodes. I will then retrieve the Latitude and Longitude Information using the pgeocode library.
    Next, I will use the latitude and longitude information to gather all the venue information from those three zones using the FourSquare database. 

## 1. Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

!pip install folium

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install geocoder # to get coordinates

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 92kB 16.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 11.0MB/s ta 0:00:01
Libraries imported.


## 2. Get Latitude and Longitude from Zip Codes

In [2]:
!pip install pgeocode #For getting coordinates from zip codes

In [3]:
import pgeocode

nomi = pgeocode.Nominatim('us')
nomi.query_postal_code("36079") #Test Run before entering into a dataframe.

postal_code         36079
country code           US
place_name           Troy
state_name        Alabama
state_code             AL
county_name          Pike
county_code           109
community_name        NaN
community_code        NaN
latitude          31.7595
longitude        -86.0107
accuracy                4
Name: 0, dtype: object

In [4]:
Zip_Codes=nomi.query_postal_code(["36079","36081","36082"])
Zip_Codes.head()

,postal_code,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,36079,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7595,-86.0107,4.0
1,36081,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7945,-85.9655,4.0
2,36082,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.8028,-85.9546,4.0


## 3. Create a Map of Troy, AL

In [5]:
# get the coordinates of Troy Alabama
address = 'Troy, Alabama'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Troy, Alabama {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Troy, Alabama 31.8087678, -85.969951.


In [6]:
# create map of Troy, Al using latitude and longitude values
map_troy = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Zip_Codes['latitude'], Zip_Codes['longitude'], Zip_Codes['postal_code']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_troy)  
    
map_troy

# By looking at the map, we can now see the the general locations of these three zip code zones. Now we have to plot the existing restaurants that exist within those zones.

In [7]:
# save the map as HTML file
map_troy.save('map_troy.html')

# 4. Connect to FourSquare API to Discover Venue Information for the Zones.

In [8]:
CLIENT_ID = 'HAPT14TCH1UXJLIVL0EERBZFXJKPOR2FWVWEQMISAGY5N0C2' #Foursquare ID
CLIENT_SECRET = 'TZWYKO4FGTK3ZQRJBI2FIP2R2UXUEFFU0KVQBPZLXCTK3QON' #Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HAPT14TCH1UXJLIVL0EERBZFXJKPOR2FWVWEQMISAGY5N0C2
CLIENT_SECRET:TZWYKO4FGTK3ZQRJBI2FIP2R2UXUEFFU0KVQBPZLXCTK3QON


In [9]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Zip_Codes['latitude'], Zip_Codes['longitude'], Zip_Codes['postal_code']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [10]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Zip Code', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(67, 7)


,Zip Code,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,36079,31.7595,-86.0107,Pike Co Lake,31.752357,-85.995704,Lake
1,36079,31.7595,-86.0107,Wayne Farms - Hatchery,31.773894,-86.005917,Food
2,36081,31.7945,-85.9655,Firehouse Subs,31.784641,-85.962677,Sandwich Place
3,36081,31.7945,-85.9655,Barnes & Noble,31.800762,-85.956268,Bookstore
4,36081,31.7945,-85.9655,Chick-fil-A,31.782618,-85.955896,Fast Food Restaurant


In [11]:
venues_df.groupby(["Zip Code"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Zip Code,,,,,,
36079,2,2,2,2,2,2
36081,52,52,52,52,52,52
36082,13,13,13,13,13,13


In [12]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 27 uniques categories.


In [13]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Lake', 'Food', 'Sandwich Place', 'Bookstore',
       'Fast Food Restaurant', 'Southern / Soul Food Restaurant',
       'Wine Bar', 'Grocery Store', 'Mexican Restaurant', 'Restaurant',
       'Japanese Restaurant', 'Coffee Shop', 'Pharmacy', 'Hotel',
       'Discount Store', 'Bar', 'Gym / Fitness Center', 'Deli / Bodega',
       'Pizza Place', 'Shipping Store', 'American Restaurant',
       'Fried Chicken Joint', 'Seafood Restaurant', 'Breakfast Spot',
       'Gas Station', 'Chinese Restaurant', 'Ice Cream Shop'],
      dtype=object)

In [14]:
# check if the results contain "Restaurant"
"Restaurant" in venues_df['VenueCategory'].unique()

True

# 5. Let's Examine the Venues in the Three Zones

In [15]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Zip Code'] = venues_df['Zip Code'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(67, 28)


,Zip Code,American Restaurant,Bar,Bookstore,Breakfast Spot,Chinese Restaurant,Coffee Shop,Deli / Bodega,Discount Store,Fast Food Restaurant,Food,Fried Chicken Joint,Gas Station,Grocery Store,Gym / Fitness Center,Hotel,Ice Cream Shop,Japanese Restaurant,Lake,Mexican Restaurant,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shipping Store,Southern / Soul Food Restaurant,Wine Bar
0,36079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,36079,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,36081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,36081,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,36081,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Group by Zip Code

In [16]:
kl_grouped = kl_onehot.groupby(["Zip Code"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(3, 28)


,Zip Code,American Restaurant,Bar,Bookstore,Breakfast Spot,Chinese Restaurant,Coffee Shop,Deli / Bodega,Discount Store,Fast Food Restaurant,Food,Fried Chicken Joint,Gas Station,Grocery Store,Gym / Fitness Center,Hotel,Ice Cream Shop,Japanese Restaurant,Lake,Mexican Restaurant,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shipping Store,Southern / Soul Food Restaurant,Wine Bar
0,36079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,36081,0.057692,0.019231,0.019231,0.019231,0.019231,0.019231,0.019231,0.038462,0.173077,0.0,0.057692,0.019231,0.038462,0.019231,0.076923,0.019231,0.019231,0.0,0.057692,0.076923,0.057692,0.019231,0.057692,0.038462,0.019231,0.019231,0.019231
2,36082,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,0.076923,0.153846,0.000000,0.0,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.0,0.076923,0.076923,0.076923,0.076923,0.000000,0.000000,0.076923,0.000000,0.076923


In [17]:
len(kl_grouped[kl_grouped["Restaurant"] > 0])

2

# 6. Create a New Dataframe with only the Relevant Information (Food Venues)

In [23]:
k1_food = kl_grouped[["Zip Code","Restaurant","American Restaurant","Breakfast Spot", "Chinese Restaurant", "Deli / Bodega", "Fast Food Restaurant", "Food", "Fried Chicken Joint", "Japanese Restaurant", "Mexican Restaurant", "Pizza Place", "Sandwich Place", "Seafood Restaurant", "Southern / Soul Food Restaurant"]]

In [28]:
k1_food.head()

,Zip Code,Restaurant,American Restaurant,Breakfast Spot,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Fried Chicken Joint,Japanese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,Southern / Soul Food Restaurant
0,36079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,36081,0.019231,0.057692,0.019231,0.019231,0.019231,0.173077,0.0,0.057692,0.019231,0.057692,0.057692,0.057692,0.038462,0.019231
2,36082,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000


# 7. Use K-Means Clustering to cluster the restaurants into the three zones.

In [29]:
# set number of clusters
kclusters = 3

kl_clustering = k1_food.drop(["Zip Code"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1], dtype=int32)

In [30]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = k1_food.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [35]:
kl_merged.rename(columns={"Zip Code": "Zip Code"}, inplace=True)
kl_merged.head()

,Zip Code,Restaurant,American Restaurant,Breakfast Spot,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Fried Chicken Joint,Japanese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,Southern / Soul Food Restaurant,Cluster Labels
0,36079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,36081,0.019231,0.057692,0.019231,0.019231,0.019231,0.173077,0.0,0.057692,0.019231,0.057692,0.057692,0.057692,0.038462,0.019231,2
2,36082,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,1


In [36]:
# merge to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(Zip_Codes.set_index("postal_code"), on="Zip Code")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(3, 27)


,Zip Code,Restaurant,American Restaurant,Breakfast Spot,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Fried Chicken Joint,Japanese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,Southern / Soul Food Restaurant,Cluster Labels,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,36079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7595,-86.0107,4.0
1,36081,0.019231,0.057692,0.019231,0.019231,0.019231,0.173077,0.0,0.057692,0.019231,0.057692,0.057692,0.057692,0.038462,0.019231,2,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7945,-85.9655,4.0
2,36082,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,1,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.8028,-85.9546,4.0


In [45]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(3, 27)


,Zip Code,Restaurant,American Restaurant,Breakfast Spot,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Fried Chicken Joint,Japanese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,Southern / Soul Food Restaurant,Cluster Labels,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,36079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7595,-86.0107,4.0
2,36082,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,1,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.8028,-85.9546,4.0
1,36081,0.019231,0.057692,0.019231,0.019231,0.019231,0.173077,0.0,0.057692,0.019231,0.057692,0.057692,0.057692,0.038462,0.019231,2,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7945,-85.9655,4.0


In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['latitude'], kl_merged['longitude'], kl_merged['Zip Code'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
for lat, lon, poi, cluster in zip(kl_merged['latitude'], kl_merged['longitude'], kl_merged['Zip Code'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [40]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [41]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Zip Code,Restaurant,American Restaurant,Breakfast Spot,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Fried Chicken Joint,Japanese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,Southern / Soul Food Restaurant,Cluster Labels,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,36079,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7595,-86.0107,4.0


In [42]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Zip Code,Restaurant,American Restaurant,Breakfast Spot,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Fried Chicken Joint,Japanese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,Southern / Soul Food Restaurant,Cluster Labels,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
2,36082,0.076923,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.076923,0.076923,0.0,0.0,0.0,1,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.8028,-85.9546,4.0


In [43]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Zip Code,Restaurant,American Restaurant,Breakfast Spot,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Fried Chicken Joint,Japanese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,Southern / Soul Food Restaurant,Cluster Labels,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
1,36081,0.019231,0.057692,0.019231,0.019231,0.019231,0.173077,0.0,0.057692,0.019231,0.057692,0.057692,0.057692,0.038462,0.019231,2,US,Troy,Alabama,AL,Pike,109.0,NaN,NaN,31.7945,-85.9655,4.0


## 8. Final Observations


By merging these restuarnts into the Zip Code clusters, I am able to see the locations that are populated by most of the city's restaurants. The most restaurants are located in the cluster for the Zip Code 36081. The second greatest amount of restaurants are located in the Zip Code cluster of 36082. The zip code with the least amount of restaurants is 36079. With this knowledge, the new restaurant owner will be better equipped to make a dicision on where his new restaurant might thrive best.